# ISIS TS1 Current Moderator Hydrogen gas and Polymer Production
## Initial Splitting

This is the hydrogen gas and polymer production for the first split of the TS1 Current moderator. It was initially split into 6 sections.
- Includes results for neutrons and photons
- Dwell times here were the percentages and averages from the powerpoint by Nathan O’Donoghue 2018 for the 'fine' simulation

Evans paper : Evans, D., 1995. Irradiation effects in liquid methane used as a neutron moderator. Cryogenics, 35(11), pp.763-766.

In [1]:
import numpy as np
import pandas as pd

from Polymer_Production_Functions import moles_per_MGy_per_g
from Polymer_Production_Functions import radicals_per_MGy_per_g
from Polymer_Production_Functions import vol_of_H_per_s
from Polymer_Production_Functions import rate_of_pol_form
from Polymer_Production_Functions import molecular_weight
from Polymer_Production_Functions import MeV_per_g_per_sec_to_MGy_per_sec

# Moles of Hydrogen gas and Polymer per MGy per gram of CH4, Number of Hydrogen and Polymer Radicals per MGy per gram of CH4, Molecular weight of Polymer
- G-Values for CH4 in umol/J, taken from Evans paper.
     - G-Values in evans paper are the yield in solid methane, g values in liquid are unknown
     - though i would say initial moles etc are likely very similar in liquid and solid, the main difference being the rate of formation
- Molecular weight - in Evans paper the polymer is said to be (CH2)n where n is the chain length. It is thought the chain length does not exceed n=20 so this is for the maximum chain length.

In [43]:
moles_H_per_MGyg = moles_per_MGy_per_g(0.66e-6)
print (moles_H_per_MGyg, 'moles of Hydrogen gas per MGy per gram of CH4')
print (radicals_per_MGy_per_g(moles_H_per_MGyg), 'radicals/g.MGy of Hydrogen')

moles_Pol_per_MGyg = moles_per_MGy_per_g(0.22e-6)
print (moles_Pol_per_MGyg, 'moles of Polymer per MGy per gram of CH4')
print (radicals_per_MGy_per_g(moles_Pol_per_MGyg), 'radicals/g.MGy of Polymer')

Mr_20 = molecular_weight(20)
print('Molecular weight of the polymer (CH2)n for the max chain length of 20 =', Mr_20)

0.00066 moles of Hydrogen gas per MGy per gram of CH4
3.97518e+20 radicals/g.MGy of Hydrogen
0.00022 moles of Polymer per MGy per gram of CH4
1.32506e+20 radicals/g.MGy of Polymer
Molecular weight of the polymer (CH2)n for the max chain length of 20 = 280


# TS1 Current Moderator

## Setting up dataframe for the split moderator 
- Absorbed dose rate calculated for both photons and neutrons 
- Particles of sufficient energy can cause bond cleavage so can assume all particles produced contribute
    - however unsure of the weighting - neutrons are high LET and photons low LET, im not sure which causes more reactive species to be made or if theres even a way to find out.

In [44]:
TS1_Current_Data = {'Cell #': ['13', '14', '13000', '13001', '14000', '14001'],
                    'CH4 Vol (cc)': [7.82E+01, 7.85E+01, 9.15E+01, 6.71E+01, 9.24E+01, 6.73E+01],
                    'CH4 Mass (g)': [3.45E+01, 3.46E+01, 4.03E+01, 2.96E+01, 4.07E+01, 2.97E+01],
                    'Energy Deposited (MeV/g/s) Neutron':[2.54E+12, 2.28E+12, 1.22E+12, 7.71E+11, 1.04E+12, 6.68E+11],
                    'Energy Deposited (MeV/g/s) Photon':[2.59E+11, 2.48E+11 ,2.21E+11, 1.61E+11, 2.12E+11, 1.56E+11],
                   }

TS1_Current_df = pd.DataFrame(TS1_Current_Data, columns =['Cell #','CH4 Vol (cc)', 'CH4 Mass (g)', 
                                                            'Energy Deposited (MeV/g/s) Neutron', 
                                                            'Energy Deposited (MeV/g/s) Photon'])

TS1_Current_df['Dose Rate (MGy/s) Neutron'] = MeV_per_g_per_sec_to_MGy_per_sec(TS1_Current_df['Energy Deposited (MeV/g/s) Neutron'])
TS1_Current_df['Dose Rate (MGy/s) Photon'] = MeV_per_g_per_sec_to_MGy_per_sec(TS1_Current_df['Energy Deposited (MeV/g/s) Photon'])



TS1_Current_df

,Cell #,CH4 Vol (cc),CH4 Mass (g),Energy Deposited (MeV/g/s) Neutron,Energy Deposited (MeV/g/s) Photon,Dose Rate (MGy/s) Neutron,Dose Rate (MGy/s) Photon
0,13,78.2,34.5,2.540000e+12,2.590000e+11,0.000407,0.000041
1,14,78.5,34.6,2.280000e+12,2.480000e+11,0.000365,0.000040
2,13000,91.5,40.3,1.220000e+12,2.210000e+11,0.000195,0.000035
3,13001,67.1,29.6,7.710000e+11,1.610000e+11,0.000124,0.000026
4,14000,92.4,40.7,1.040000e+12,2.120000e+11,0.000167,0.000034
5,14001,67.3,29.7,6.680000e+11,1.560000e+11,0.000107,0.000025


## Hydrogen gas production rate
- Again calculated for each particle for the split moderator 
- 0.43 is the Specific Gravity (as referenced in the Evans paper), in the case of this function it basically acts as the density in cm3/g

In [45]:
TS1_Current_Hvol_n = vol_of_H_per_s(moles_H_per_MGyg, TS1_Current_df['CH4 Vol (cc)'], TS1_Current_df['Dose Rate (MGy/s) Neutron'], 0.43)
TS1_Current_Hvol_p = vol_of_H_per_s(moles_H_per_MGyg, TS1_Current_df['CH4 Vol (cc)'], TS1_Current_df['Dose Rate (MGy/s) Photon'], 0.43)

## Polymer production rate

In [46]:
TS1_Current_Polymer_n = rate_of_pol_form(0.22e-6, Mr_20, TS1_Current_df['Dose Rate (MGy/s) Neutron'], TS1_Current_df['CH4 Mass (g)'])
TS1_Current_Polymer_p = rate_of_pol_form(0.22e-6, Mr_20, TS1_Current_df['Dose Rate (MGy/s) Photon'], TS1_Current_df['CH4 Mass (g)'])

## Results

In [47]:
TS1_Current_df_results = pd.DataFrame(columns = ['Cell #', 
                                                 'Hydrogen Production Rate (cc/s) Neutron',
                                                 'Polymer Production Rate (g/s) Neutron',
                                                 'Hydrogen Production Rate (cc/s) Photon',
                                                 'Polymer Production Rate (g/s) Photon'])


TS1_Current_df_results['Cell #'] = TS1_Current_df['Cell #']
TS1_Current_df_results['Hydrogen Production Rate (cc/s) Neutron'] = TS1_Current_Hvol_n 
TS1_Current_df_results['Polymer Production Rate (g/s) Neutron'] = TS1_Current_Polymer_n 
TS1_Current_df_results['Hydrogen Production Rate (cc/s) Photon'] = TS1_Current_Hvol_p
TS1_Current_df_results['Polymer Production Rate (g/s) Photon'] = TS1_Current_Polymer_p 

TS1_Current_df_results

,Cell #,Hydrogen Production Rate (cc/s) Neutron,Polymer Production Rate (g/s) Neutron,Hydrogen Production Rate (cc/s) Photon,Polymer Production Rate (g/s) Photon
0,13,0.202308,0.000865,0.020629,0.000088
1,14,0.182296,0.000779,0.019829,0.000085
2,13000,0.113698,0.000485,0.020596,0.000088
3,13001,0.052692,0.000225,0.011003,0.000047
4,14000,0.097876,0.000418,0.019952,0.000085
5,14001,0.045789,0.000196,0.010693,0.000046


## Totals and checks
- here ive worked out the overall dose rate by summing the raw energies and getting the absorbed dose rate per gram (on excel) for all particles combined. Then calculated the overall dose rate using this value (below) and found the volume of hydrogen gas and the rate of polymer produced in the moderator as a whole for all particles, the whole volume of methane and the whole mass of methane.
### Total absorbed dose per gram (MeV/g/s) for all particles = 1.65E+12
### Total volume = 4.75E+02 cc
### Total Mass = 2.09E+02 g

In [48]:
Total_MGy_per_s = MeV_per_g_per_sec_to_MGy_per_sec(1.65E+12)
print('Absorbed dose rate for all particles = ', Total_MGy_per_s, 'MGy/s')


Overall_h_production_rate = vol_of_H_per_s(moles_H_per_MGyg, 4.75E+02, Total_MGy_per_s, 0.43)
print('Hydrogen production rate for all particles, total dose rate and volume = ', Overall_h_production_rate, 'cm3/s')

Overall_pol_production_rate = rate_of_pol_form(0.22e-6, Mr_20, Total_MGy_per_s, 2.09E+02)
print('Polymer production rate for all particles, total dose rate and mass = ', Overall_pol_production_rate, 'g/s')

Absorbed dose rate for all particles =  0.0002643597 MGy/s
Hydrogen production rate for all particles, total dose rate and volume =  0.7982690096304 cm3/s
Polymer production rate for all particles, total dose rate and mass =  0.00340347252168 g/s


In [49]:
tot_h_n = TS1_Current_df_results['Hydrogen Production Rate (cc/s) Neutron'].sum()
tot_h_p = TS1_Current_df_results['Hydrogen Production Rate (cc/s) Photon'].sum()

print(tot_h_n, tot_h_p)
print('Total hydrogen production rate for all particles, using split data = ', tot_h_n + tot_h_p, 'cm3/s') 
#ever so slightly different to total from total dose rate but likely just rounding 

tot_pol_n = TS1_Current_df_results['Polymer Production Rate (g/s) Neutron'].sum()
tot_pol_p = TS1_Current_df_results['Polymer Production Rate (g/s) Photon'].sum()
print(tot_pol_n, tot_pol_p)
print('Total polymer production rate for all particles, using split data = ', tot_pol_n + tot_pol_p, 'g/s')
#again ever so slightly different to total from total dose rate but likely just rounding

0.6946590394368981 0.10270195510950145
Total hydrogen production rate for all particles, using split data =  0.7973609945463995 cm3/s
0.0029674727394681604 0.00043869413627424
Total polymer production rate for all particles, using split data =  0.0034061668757424004 g/s


# Dwell Times

## Percentages and Maximum
First using the percentages for the total CH4 volume > 1s and >4s 
- Total > 1s = 85%
- Total > 4s = 58%


### Greater than 1s
- greater than 1 but up to the maximum of 6.8 (i think)
- 85% of the volume is greater than 1s, so same as 85% of the mass right? (worked out mass from vol using the 1cc value on the excel sheet rather than just doing 85% of the mass)
- use the total absorbed dose rate as it accounts for the volume/mass

In [74]:
gt_1s_vol_CH4 = 4.75E+02 * 0.85
print('85% the volume of CH4', gt_1s_vol_CH4)

gt_1s_mass_CH4 = gt_1s_vol_CH4 * 4.41E-01
print('mass of CH4 in 85% of the volume', gt_1s_mass_CH4)


85% the volume of CH4 403.75
mass of CH4 in 85% of the volume 178.05375


In [79]:
gt_1s_pol_rate = rate_of_pol_form(0.22e-6, Mr_20, Total_MGy_per_s, gt_1s_mass_CH4)
print('polymer production rate for 85% of the volume of CH4 =', gt_1s_pol_rate, 'g/s')

for i in np.linspace(1, 6.8, num=30):
    print(gt_1s_pol_rate * i, 'grams of polymer produced for', i, 'seconds')


polymer production rate for 85% of the volume of CH4 = 0.0028995265335267 g/s
0.0028995265335267 grams of polymer produced for 1.0 seconds
0.00347943184023204 grams of polymer produced for 1.2 seconds
0.00405933714693738 grams of polymer produced for 1.4 seconds
0.004639242453642721 grams of polymer produced for 1.6 seconds
0.00521914776034806 grams of polymer produced for 1.7999999999999998 seconds
0.0057990530670534 grams of polymer produced for 2.0 seconds
0.006378958373758741 grams of polymer produced for 2.2 seconds
0.00695886368046408 grams of polymer produced for 2.4 seconds
0.0075387689871694195 grams of polymer produced for 2.5999999999999996 seconds
0.00811867429387476 grams of polymer produced for 2.8 seconds
0.008698579600580101 grams of polymer produced for 3.0 seconds
0.00927848490728544 grams of polymer produced for 3.1999999999999997 seconds
0.00985839021399078 grams of polymer produced for 3.4 seconds
0.01043829552069612 grams of polymer produced for 3.5999999999999996

### Greater than 4s
- again up to the maximum of 6.8s

In [76]:
gt_4s_vol_CH4 = 4.75E+02 * 0.58
print('58% the volume of CH4', gt_4s_vol_CH4)

gt_4s_mass_CH4 = gt_4s_vol_CH4 * 4.41E-01
print('mass of CH4 in 58% of the volume', gt_4s_mass_CH4)


58% the volume of CH4 275.5
mass of CH4 in 58% of the volume 121.4955


In [80]:
gt_4s_pol_rate = rate_of_pol_form(0.22e-6, Mr_20, Total_MGy_per_s, gt_4s_mass_CH4)
print('polymer production rate for 58% of the volume of CH4 =', gt_4s_pol_rate, 'g/s')

for j in np.linspace(4, 6.8, num=30):
    print(gt_1s_pol_rate * j, 'grams of polymer produced for', j, 'seconds')

polymer production rate for 58% of the volume of CH4 = 0.00197850045817116 g/s
0.0115981061341068 grams of polymer produced for 4.0 seconds
0.011878060420102482 grams of polymer produced for 4.096551724137931 seconds
0.012158014706098164 grams of polymer produced for 4.1931034482758625 seconds
0.012437968992093845 grams of polymer produced for 4.289655172413793 seconds
0.012717923278089526 grams of polymer produced for 4.386206896551724 seconds
0.012997877564085207 grams of polymer produced for 4.482758620689655 seconds
0.01327783185008089 grams of polymer produced for 4.5793103448275865 seconds
0.01355778613607657 grams of polymer produced for 4.675862068965517 seconds
0.013837740422072252 grams of polymer produced for 4.772413793103448 seconds
0.014117694708067934 grams of polymer produced for 4.86896551724138 seconds
0.014397648994063615 grams of polymer produced for 4.9655172413793105 seconds
0.014677603280059296 grams of polymer produced for 5.062068965517241 seconds
0.01495755756

### Maximum and average
* per one cycle of the methane
* for the moderator as a whole

In [54]:
max_pol_produced = Overall_pol_production_rate * 6.8
avg_pol_produced = Overall_pol_production_rate * 2.6

print('maximum polymer produced per methane cycle =',max_pol_produced, 'grams')
print('average polymer produced per methane cycle =',avg_pol_produced, 'grams')

maximum polymer produced per methane cycle = 0.023143613147423997 grams
average polymer produced per methane cycle = 0.008849028556368 grams
